In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Resize Images

In [2]:
from PIL import Image
import sys

def resize(path,mw):
    dirs = os.listdir(path)
    outpath = path.replace("input", "output") + "resized/"
    os.makedirs(outpath, exist_ok=True)
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((512,512), Image.ANTIALIAS)
            imResize.convert('RGB').save(outpath+item, 'JPEG', quality=90)

resize("/kaggle/input/men-women-classification/men/", "men/")
resize("/kaggle/input/men-women-classification/women/", "women/")

/tmp/ipykernel_20/2538394186.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  imResize = im.resize((512,512), Image.ANTIALIAS)


## Define Network

Network consists of two convolutional layers followed by three linear layers.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 3 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 6, 10)
        self.conv2 = nn.Conv2d(6, 16, 10)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 121 * 121, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.normalize(x)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(10, 10), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(10, 10), stride=(1, 1))
  (fc1): Linear(in_features=234256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


## Define Image Dataset

In [4]:
from torch.utils.data import Dataset, ConcatDataset, DataLoader, random_split
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, label, img_dir, transform=None, target_transform=None):
        self.img_label = label
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        count = 0
        # Iterate directory
        for path in os.listdir(self.img_dir):
            # check if current path is a file
            if os.path.isfile(os.path.join(self.img_dir, path)):
                count += 1
        return count

    def __getitem__(self, idx):
        img_path = os.listdir(self.img_dir)[idx]
        image = read_image(os.path.join(self.img_dir,img_path))
        label = self.img_label
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [5]:
men_ds = CustomImageDataset(0, "/kaggle/output/men-women-classification/men/resized")
women_ds = CustomImageDataset(1, "/kaggle/output/men-women-classification/women/resized")

In [6]:
dataset = ConcatDataset([men_ds, women_ds])
train_ds, val_ds, test_ds = random_split(dataset, [0.7, 0.2, 0.1])
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=1)
test_loader = DataLoader(test_ds)

## Train Network

Manually propagate data through network and back-propagate loss.

In [7]:
import torch.optim as optim

# create loss criterion
criterion = nn.CrossEntropyLoss()

# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=0.0001)

# in your training loop:
for i, data in enumerate(train_loader, 0):
    input, target = data
    input = input.float()
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()    # Does the update

Save trained network.

In [8]:
torch.save(net.state_dict(), "/kaggle/output/net.pth")

In [9]:
newnet = Net()
newnet.load_state_dict(torch.load("/kaggle/output/net.pth"))

<All keys matched successfully>

In [10]:
data, target = next(iter(val_loader))

## Validation

Manually propagate validation data through network and determine accuracy based on true target values.

In [11]:
matches = 0
for i, data in enumerate(val_loader, 0):
    input, target = data
    input = input.float()
    output = net(input)
    output = torch.argmax(output)
    if target == output:
        matches += 1
print("Accuracy: ", matches/len(val_loader))

Accuracy:  0.5465465465465466


We get accuracy slightly better than random chance.

In [12]:
import matplotlib.pyplot as plt
input, target = data
input = input.float()
F.normalize(input,0)[0]

tensor([[[ 0.6667,  0.6667,  0.6667,  ...,  0.6667,  0.6667,  0.6667],
         [ 0.6667,  0.6667,  0.6667,  ...,  0.6667,  0.6667,  0.6667],
         [ 0.6667,  0.6667,  0.6667,  ...,  0.6667,  0.6667,  0.6667],
         ...,
         [10.0000, 10.6667, 11.3333,  ...,  0.6667,  0.6667,  0.6667],
         [13.6667, 14.0000,  9.6667,  ...,  0.6667,  0.6667,  0.6667],
         [10.6667, 15.0000, 13.0000,  ...,  0.6667,  0.6667,  0.6667]],

        [[ 0.6667,  0.6667,  0.6667,  ...,  0.6667,  0.6667,  0.6667],
         [ 0.6667,  0.6667,  0.6667,  ...,  0.6667,  0.6667,  0.6667],
         [ 0.6667,  0.6667,  0.6667,  ...,  0.6667,  0.6667,  0.6667],
         ...,
         [10.0000, 10.6667, 11.3333,  ...,  0.6667,  0.6667,  0.6667],
         [13.6667, 14.0000,  9.6667,  ...,  0.6667,  0.6667,  0.6667],
         [10.6667, 15.0000, 13.0000,  ...,  0.6667,  0.6667,  0.6667]],

        [[ 0.6667,  0.6667,  0.6667,  ...,  0.6667,  0.6667,  0.6667],
         [ 0.6667,  0.6667,  0.6667,  ...,  0